Data Cleaning

In [1]:
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)



import datetime
def validate(date_text):
        try:
            if datetime.date.fromisoformat(date_text):
                  return True
        except ValueError:
              return date_text
#%%
class DataCleaning():
    def __init__(self,dataframe):
        self.dataframe = dataframe


    def clean_user_data(self):
        clean_1_no_null = self.dataframe[self.dataframe.country_code != 'NULL']
        messy_first_names = []
        for item in clean_1_no_null.first_name:
            if has_numbers(item):
                messy_first_names.append(item)
        clean_2_users = clean_1_no_null[~clean_1_no_null.first_name.isin(messy_first_names)]
        clean_3_country_code = clean_2_users.replace('GGB','GB')
        cleaned_data = clean_3_country_code
        return cleaned_data

    
    def clean_card_data(self):
        remove_null = self.dataframe[self.dataframe.card_number != 'NULL']
        date_errors = []
        for item in remove_null.expiry_date:
            if item.isalpha():
                date_errors.append(item)
        remove_date_error = remove_null.copy()

        for item in date_errors:
            remove_date_error.drop(remove_date_error.loc[remove_date_error['expiry_date']==item].index, inplace=True)
        date_format_errors = []
        for item in self.dataframe.date_payment_confirmed:
             if validate(item):
                  pass
             else:
                  date_format_errors.append(item)

        clean_date_format = remove_date_error.copy()
        for item in date_format_errors:
             clean_date_format.drop(clean_date_format.loc[clean_date_format['date_payment_confirmed']==item].index, inplace=True)

        return clean_date_format

Data Extractor

In [12]:
import pandas as pd
import tabula as tb




class DataExtractor():
    def __init__(self, database_connector_instance = None, table_name = None):
        self.database_connector_instance = database_connector_instance
        self.table_name = table_name

    def read_df_tables(self):
        table_list = self.database_connector_instance.list_db_tables()
        if self.table_name in table_list:
            table = pd.read_sql_table(self.table_name, self.database_connector_instance.init_db_engine())
            return table
        else:
            return 'Table not in Database'


    def retrieve_pdf_data(self,link):
        pdf_data = tb.read_pdf(link,pages = 'all')
        df_pdf = pd.concat(pdf_data)
        return df_pdf

Data Utils

In [5]:




import yaml
from yaml.loader import SafeLoader
from sqlalchemy import create_engine, inspect





class DatabaseConnector():
    def __init__(self,file_name = None):
        self.file_name = file_name

    def read_db_creds(self):
        with open(self.file_name) as f:
            data = yaml.load(f, Loader=SafeLoader)
            return data

    def init_db_engine(self):
        data_2 = self.read_db_creds()
        engine = create_engine(f"postgresql+psycopg2://{data_2['RDS_USER']}:{data_2['RDS_PASSWORD']}@{data_2['RDS_HOST']}:{data_2['RDS_PORT']}/{data_2['RDS_DATABASE']}")
        return engine

    def list_db_tables(self):
        inspector = inspect(self.init_db_engine())
        return inspector.get_table_names()

    def upload_to_db(self,dataframe,table_name):
        DATABASE_TYPE = 'postgresql'
        DBAPI = 'psycopg2'
        HOST = 'localhost'
        USER = 'postgres'
        PASSWORD = 'P0037979'
        DATABASE = 'Sales_Data'
        PORT = 5432
        engine_2 = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")
        dataframe.to_sql(table_name,engine_2,if_exists = 'replace')




create connection

In [6]:
con = DatabaseConnector()

extract data from pdf

In [14]:
ext = DataExtractor()
ext.retrieve_pdf_data('https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf')

,card_number,expiry_date,card_provider,date_payment_confirmed
0,30060773296197,09/26,Diners Club / Carte Blanche,2015-11-25
1,349624180933183,10/23,American Express,2001-06-18
2,3529023891650490,06/23,JCB 16 digit,2000-12-26
3,213142929492281,09/27,JCB 15 digit,2011-02-12
4,502067329974,10/25,Maestro,1997-03-13
...,...,...,...,...
4,180036921556789,12/28,JCB 15 digit,1997-06-06
5,180018030448512,11/24,JCB 15 digit,2004-06-16
6,3569953313547220,04/24,JCB 16 digit,2020-02-05
7,4444521712606810,06/27,VISA 16 digit,2008-06-16


In [15]:
pdf_data = ext.retrieve_pdf_data('https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf')

In [17]:
clean = DataCleaning(pdf_data)

In [18]:
clean_data = clean.clean_card_data()

In [19]:
clean_data.head()

,card_number,expiry_date,card_provider,date_payment_confirmed
0,30060773296197,09/26,Diners Club / Carte Blanche,2015-11-25
1,349624180933183,10/23,American Express,2001-06-18
2,3529023891650490,06/23,JCB 16 digit,2000-12-26
3,213142929492281,09/27,JCB 15 digit,2011-02-12
4,502067329974,10/25,Maestro,1997-03-13


In [20]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14992 entries, 0 to 8
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             14992 non-null  object
 1   expiry_date             14992 non-null  object
 2   card_provider           14992 non-null  object
 3   date_payment_confirmed  14992 non-null  object
dtypes: object(4)
memory usage: 585.6+ KB


In [21]:
con.upload_to_db(clean_data,'dim_card_details')